# Integrate multoprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from numpy import sin, cos, pi, exp, sqrt
import os
import ipyparallel as ipp
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)



### Start client

In [2]:
## connect to ipcluster using default arguments
ipyclient = ipp.Client()

## count how many engines are connected
print(len(ipyclient), 'cores')
lbview = ipyclient.load_balanced_view()

8 cores


# Routines

In [3]:
def ServiceZone(Lat, Lon, GS_X, GS_Y, h):
    k = len(Lat)
    l = len(Lat[0])
    E = np.array([[1] * k] * l)
    for i in range(len(GS_X)):
        filt_GS_X = GetClearList(GS_X, [i])
        filt_GS_Y = GetClearList(GS_Y, [i])
        HDOP, VDOP =  GetHVDOP(Lat, Lon, filt_GS_X, filt_GS_Y)
        class_HDOP = BinByDOP(HDOP)
        E = E * class_HDOP
    return E

In [4]:
def ServiceZonePlot(Lat, Lon, GS_X, GS_Y, h):
    k = len(Lat)
    l = len(Lat[0])
    E = np.array([[1] * k] * l)
    for i in range(len(GS_X)):
        filt_GS_X = GetClearList(GS_X, [i])
        filt_GS_Y = GetClearList(GS_Y, [i])
        HDOP, VDOP =  GetHVDOP(Lat, Lon, filt_GS_X, filt_GS_Y)
        class_HDOP = BinByDOP(HDOP)
        E = E * class_HDOP
        
        %matplotlib inline
        fig = plt.figure(figsize=(15, 10))
        ax = fig.add_subplot()
        pt = ax.contourf(Lon, Lat, class_HDOP, zdir='z', offset= -10)
        ax.scatter(filt_GS_Y, filt_GS_X,  linewidths = 10, color = 'black', marker = '^')
        fig.colorbar(pt)
        ax.set_title('GPS HDOP sigma = 12'  + '; Cells = ' + str(CalcSquare(HDOP)) + '; Square = ' +  str(round(CalcRealSquare(Lat, Lon, HDOP, h),2)) + ' km^2')
        ax.xaxis.set_minor_locator(MultipleLocator(0.5))
        ax.yaxis.set_minor_locator(MultipleLocator(0.5))
        ax.grid('minor')
        plt.show()
        
    return E

In [5]:
def GetClearList(l, s):
    new_ = l.copy()
    if(len(s) > 0):
        new_ = np.delete(new_, s)
    return new_.tolist()

In [6]:
def GetGSList(l, s):
    new = []
    for i in s:
        new.append(l[i])
    return new

In [7]:
def geodetic2ecef(lat, lon, alt):
    lat = lat * pi / 180
    lon = lon * pi / 180
    a  = 6378137.0
    b = 6356752.314
   # radius of curvature of the prime vertical section
    N = a ** 2 / sqrt(
        a ** 2 * cos(lat) ** 2 + b ** 2 * sin(lat) ** 2
    )
    # Compute cartesian (geocentric) coordinates given  (curvilinear) geodetic
    # coordinates.
    x = (N + alt) * cos(lat) * cos(lon)
    y = (N + alt) * cos(lat) * sin(lon)
    z = (N * (b / a) ** 2 + alt) * sin(lat)
    return x, y ,z

In [8]:
def OneLonToKm(lat):
    return 111.3 * cos(lat * pi / 180)

def OneLatToKm():
    return 111.3 

In [9]:
def FromGeoToTOP(lat, lon):
    N = np.array([[0.] * 3] * 3)
    sin_lat = sin(lat)
    cos_lat = cos(lat)
    sin_lon = sin(lon)
    cos_lon = cos(lon)
    N[0][0] = -sin_lon
    N[0][1] = cos_lon
    N[0][2] = 0.0
    N[1][0] = (-sin_lat) * cos_lon
    N[1][1] = (-sin_lat) * sin_lon
    N[1][2] = cos_lat
    N[2][0] = cos_lat * cos_lon
    N[2][1] = cos_lat * sin_lon
    N[2][2] = sin_lat
    return N

In [10]:
def ClassificationRow(a):
    if(a > 50):
        return 0
    else:
        return 1


def Classification(a):
    return list(map(ClassificationRow, a))

def CalcSquare(DOP):
    return np.array(list(map(Classification, DOP))).sum()

def BinByDOP(DOP):
    return np.array(list(map(Classification, DOP)))

def CalcRealSquareByBinMat(Lat, Lon, DOP, h = 0.1):
    k = len(Lat[0])
    l = len(Lon)
    sum_sqr = 0
    for i in range(0, k):
        for j in range(0, l): 
            if(DOP[i][j] > 0):
                if((i + 1) != k):
                    Lat_mean = ((Lat[i + 1][j] + Lat[i][j]) / 2)
                else:
                    Lat_mean = Lat[i][j]
                sum_sqr += (h * h * OneLatToKm() * OneLonToKm(Lat_mean))
    return sum_sqr


def CalcRealSquare(Lat, Lon, DOP, h = 0.1):
    k = len(Lat[0])
    l = len(Lon)
    sum_sqr = 0
    for i in range(0, k):
        for j in range(0, l): 
            if(DOP[i][j] <= 50):
                if((i + 1) != k):
                    Lat_mean = ((Lat[i + 1][j] + Lat[i][j]) / 2)
                else:
                    Lat_mean = Lat[i][j]
                sum_sqr += (h * h * OneLatToKm() * OneLonToKm(Lat_mean))
    return sum_sqr

def CalcRealSquareHV(Lat, Lon, HDOP, VDOP, h = 0.1):
    k = len(Lat[0])
    l = len(Lon)
    vsum_sqr = 0
    hsum_sqr = 0
    for i in range(0, k):
        for j in range(0, l): 
            if(HDOP[i][j] <= 50):
                if((i + 1) != k):
                    Lat_mean = ((Lat[i + 1][j] + Lat[i][j]) / 2)
                else:
                    Lat_mean = Lat[i]
                hsum_sqr += (h * h * OneLatToKm() * OneLonToKm(Lat_mean))
            if(VDOP[i][j] <= 50):
                if((i + 1) != k):
                    Lat_mean = ((Lat[i + 1][j] + Lat[i][j]) / 2)
                else:
                    Lat_mean = Lat[i]
                vsum_sqr += (h * h * OneLatToKm() * OneLonToKm(Lat_mean))
    return hsum_sqr, vsum_sqr

def GetDiffBetweenGS(GS_Lats):
    return abs(GS_Lats[0] - GS_Lats[1]) * OneLatToKm()

def PlotVDOP(i):
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot()
    pt = ax.contour(Lat, Lon, VDOP, zdir='z', offset= -10, cmap=cm.jet, levels = 12)
    ax.clabel(pt, pt.levels, inline=True, fontsize=10)
    ax.scatter(GS_X, GS_Y, linewidths = 10,  color = 'black', marker = '^')
    fig.colorbar(pt)
    ax.set_title('GPS VDOP sigma = 12')
    ax.xaxis.set_minor_locator(MultipleLocator(0.5))
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))
    ax.grid('minor')
    plt.savefig('VDOP{0}.png'.format(i))
    plt.show()

In [11]:
def IsDetectMod(ro, h, k = 4/3):
    if(h >= (0.0785 * ro / k)):
        return True
    else:
        return False

In [12]:
def IsDetect(d, h1, h2, k = 3.57):
    if(d <= (k * (h1**(1/2) + h2**(1/2)))):
        return True
    else:
        return False

In [13]:
def GetHVDOP(Lat, Lon, GS_X, GS_Y, h = 6000, sigma = 12):
    n_GS = len(GS_X)
    k = len(Lat[0])
    l = len(Lat)
   
    HDOP = np.array([[0] * k] * l)
    VDOP = np.array([[0] * k] * l)
    for i in range(0, k):
        for j in range(0, l):  
            x_, y_, z_ =  geodetic2ecef(Lat[i][j], Lon[i][j], h )
            AC = np.array([x_, y_, z_])
            A = []
            for r in range(0, n_GS):
                x_, y_, z_ = geodetic2ecef(GS_X[r], GS_Y[r], 20)
                GS = np.array([x_, y_, z_])
                nrm = np.linalg.norm(GS - AC)
                if(IsDetect(nrm / 1000, 20, h)):
                    AC_GS = FromGeoToTOP(Lat[i][j] * pi / 180, Lon[i][j] * pi / 180) @ (GS - AC).T / nrm
                    A.append([AC_GS[0], AC_GS[1], AC_GS[2]  , 1])
                
            if(len(A) >= 4):
                A = np.array(A)
                Q = sigma**2 * np.linalg.pinv(np.dot(A.T, A))
                HDOP[i][j] = np.sqrt(Q[0, 0] + Q[1, 1])
                VDOP[i][j] = np.sqrt(Q[2, 2])
                if(HDOP[i][j] > 500):
                    HDOP[i][j] = 500
                if(VDOP[i][j] > 500):
                    VDOP[i][j] = 500
            else:
                HDOP[i][j] = 500
                VDOP[i][j] = 500
    return HDOP, VDOP

In [14]:
def GetHVGDOP(Lat, Lon, GS_X, GS_Y, h = 6000, sigma = 12):
    n_GS = len(GS_X)
    k = len(Lat[0])
    l = len(Lat)
    HDOP = np.array([[0] * k] * l)
    VDOP = np.array([[0] * k] * l)
    GDOP = np.array([[0] * k] * l)
    H = [200, 250, 270, 260, 225]
    for i in range(0, k):
        for j in range(0, l):  
            x_, y_, z_ =  geodetic2ecef(Lat[i][j], Lon[i][j], h )
            AC = np.array([x_, y_, z_])
            A = []
            for r in range(0, n_GS):
                x_, y_, z_ = geodetic2ecef(GS_X[r], GS_Y[r], H[r])
                GS = np.array([x_, y_, z_])
                nrm = np.linalg.norm(GS - AC)
                if(IsDetect(nrm / 1000, H[r], h) or True):
                    AC_GS = FromGeoToTOP(Lat[i][j] * pi / 180, Lon[i][j] * pi / 180) @ (GS - AC).T / nrm
                    A.append([AC_GS[0], AC_GS[1], AC_GS[2]  , 1])
                
            if(len(A) >= 4):
                A = np.array(A)
                Q = sigma**2 * np.linalg.pinv(np.dot(A.T, A))
                HDOP[i][j] = np.sqrt(Q[0, 0] + Q[1, 1])
                VDOP[i][j] = np.sqrt(Q[2, 2])
                GDOP[i][j] = np.sqrt(Q[0, 0] + Q[1, 1] + Q[2, 2] + Q[3,3])
                if(HDOP[i][j] > 500):
                    HDOP[i][j] = 500
                if(VDOP[i][j] > 500):
                    VDOP[i][j] = 500
                if(GDOP[i][j] > 500):
                    GDOP[i][j] = 500
            else:
                HDOP[i][j] = 500
                VDOP[i][j] = 500
                GDOP[i][j] = 500
    return HDOP, VDOP, GDOP

In [15]:
def GetDOP(Lat, Lon, GS_X, GS_Y, h):
    import numpy as np
    from numpy import cos, sin, pi, sqrt
        
    def CalcRealSquareHV(Lat, Lon, HDOP, VDOP, h = 0.1):
        def OneLonToKm(lat):
            return 111.3 * cos(lat * pi / 180)

        def OneLatToKm():
            return 111.3 
        k = len(Lat[0])
        l = len(Lon)
        vsum_sqr = 0
        hsum_sqr = 0
        for i in range(0, k):
            for j in range(0, l): 
                if(HDOP[i][j] <= 50):
                    if((i + 1) != k):
                        Lat_mean = ((Lat[i + 1][j] + Lat[i][j]) / 2)
                    else:
                        Lat_mean = Lat[i]
                    hsum_sqr += (h * h * OneLatToKm() * OneLonToKm(Lat_mean))
                if(VDOP[i][j] <= 50):
                    if((i + 1) != k):
                        Lat_mean = ((Lat[i + 1][j] + Lat[i][j]) / 2)
                    else:
                        Lat_mean = Lat[i]
                    vsum_sqr += (h * h * OneLatToKm() * OneLonToKm(Lat_mean))
        return hsum_sqr, vsum_sqr
    
    def FromGeoToTOP(lat, lon):
        N = np.array([[0.] * 3] * 3)
        sin_lat = sin(lat)
        cos_lat = cos(lat)
        sin_lon = sin(lon)
        cos_lon = cos(lon)
        N[0][0] = -sin_lon
        N[0][1] = cos_lon
        N[0][2] = 0.0
        N[1][0] = (-sin_lat) * cos_lon
        N[1][1] = (-sin_lat) * sin_lon
        N[1][2] = cos_lat
        N[2][0] = cos_lat * cos_lon
        N[2][1] = cos_lat * sin_lon
        N[2][2] = sin_lat
        return N
    
    def IsDetect(d, h1, h2, k = 3.57):
        if(d <= (k * (h1**(1/2) + h2**(1/2)))):
            return True
        else:
            return False
        
    def geodetic2ecef(lat, lon, alt):
        lat = lat * pi / 180
        lon = lon * pi / 180
        a  = 6378137.0
        b = 6356752.314
       # radius of curvature of the prime vertical section
        N = a ** 2 / sqrt(
            a ** 2 * cos(lat) ** 2 + b ** 2 * sin(lat) ** 2
        )
        # Compute cartesian (geocentric) coordinates given  (curvilinear) geodetic
        # coordinates.
        x = (N + alt) * cos(lat) * cos(lon)
        y = (N + alt) * cos(lat) * sin(lon)
        z = (N * (b / a) ** 2 + alt) * sin(lat)
        return x, y ,z
    
    def GetHVDOP(Lat, Lon, GS_X, GS_Y, h = 6000, sigma = 12):
        n_GS = len(GS_X)
        k = len(Lat[0])
        l = len(Lat)

        HDOP = np.array([[0] * k] * l)
        VDOP = np.array([[0] * k] * l)
        for i in range(0, k):
            for j in range(0, l):  
                x_, y_, z_ =  geodetic2ecef(Lat[i][j], Lon[i][j], h )
                AC = np.array([x_, y_, z_])
                A = []
                for r in range(0, n_GS):
                    x_, y_, z_ = geodetic2ecef(GS_X[r], GS_Y[r], 20)
                    GS = np.array([x_, y_, z_])
                    nrm = np.linalg.norm(GS - AC)
                    if(IsDetect(nrm / 1000, 20, h)):
                        AC_GS = FromGeoToTOP(Lat[i][j] * pi / 180, Lon[i][j] * pi / 180) @ (GS - AC).T / nrm
                        A.append([AC_GS[0], AC_GS[1], AC_GS[2]  , 1])

                if(len(A) >= 4):
                    A = np.array(A)
                    Q = sigma**2 * np.linalg.pinv(np.dot(A.T, A))
                    HDOP[i][j] = np.sqrt(Q[0, 0] + Q[1, 1])
                    VDOP[i][j] = np.sqrt(Q[2, 2])
                    if(HDOP[i][j] > 500):
                        HDOP[i][j] = 500
                    if(VDOP[i][j] > 500):
                        VDOP[i][j] = 500
                else:
                    HDOP[i][j] = 500
                    VDOP[i][j] = 500
        return HDOP, VDOP

    HDOP, VDOP = GetHVDOP(Lat, Lon, GS_X, GS_Y)
    hs, vs = CalcRealSquareHV(Lat, Lon, HDOP, VDOP, h)
    return hs, vs
   

# Program

In [ ]:
%matplotlib inline
X_c = 0
Y_c = 39
offset_C = 0
n = 20
step_ = 0.1
h = 0.005
Lat = np.arange(X_c - 2, X_c + 2, h)
Lon = np.arange(Y_c - 2, Y_c + 2, h) 
Lat, Lon = np.meshgrid(Lat, Lon)
max_VDOP = [0] * 2 # value - offset
max_HDOP = [0] * 2 # value - offset
delta = []
Hcells = []
Vcells = []
Hsquare = []
Vsquare = []
Zone = []
res = {}
for i in range(n):
    offset_C += step_
    #Прямая геодезическая задача
    GS_X = np.array([X_c + offset_C * cos(315 * pi / 180), X_c + offset_C * cos(225 * pi / 180), X_c, X_c + offset_C * cos(135 * pi / 180), X_c + offset_C * cos(45 * pi / 180)]) 
    GS_Y = np.array([Y_c + offset_C * sin(315 * pi / 180), Y_c + offset_C * sin(225 * pi / 180), Y_c, Y_c + offset_C * sin(135 * pi / 180), Y_c + offset_C * sin(45 * pi / 180)] ) 
    async_ = lbview.apply(GetDOP, Lat, Lon, GS_X, GS_Y, h)      
    res[i] = async_    
ipyclient.wait()
    

In [ ]:
res

In [ ]:
Hsqueare = []
Vsqueare = []
for ridx in range(n):
    print( "job: {}, result: {}".format(ridx, res[ridx].get()))
    Hsqueare.append(res[ridx].get()[0])
    Vsqueare.append(res[ridx].get()[1])
Hsqueare = np.array(Hsqueare)
Vsqueare = np.array(Vsqueare)

In [ ]:
max_H = np.argmax(Hsqueare)
max_V = np.argmax(Vsqueare)

max_VDOP = [Hsqueare[max_V], step_ * (max_V + 1)] 
max_HDOP = [Hsqueare[max_H], step_ * (max_H + 1)]

## Оптимум HDOP

In [ ]:
#Optimum HDOP
offset_C = max_HDOP[1]
GS_X = np.array([X_c + offset_C * cos(315 * pi / 180), X_c + offset_C * cos(225 * pi / 180), X_c, X_c + offset_C * cos(135 * pi / 180), X_c + offset_C * cos(45 * pi / 180)]) 
GS_Y = np.array([Y_c + offset_C * sin(315 * pi / 180), Y_c + offset_C * sin(225 * pi / 180), Y_c, Y_c + offset_C * sin(135 * pi / 180), Y_c + offset_C * sin(45 * pi / 180)] ) 
HDOP, VDOP =  GetHVDOP(Lat, Lon, GS_X, GS_Y)

In [ ]:
%matplotlib inline
plt.scatter(GS_X, GS_Y)
plt.title('HDOP offset = ' + str(offset_C) + '; Max offset = ' + str(n * step_))

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contour(Lon, Lat, VDOP, zdir='z', offset= -10, cmap=cm.jet, levels = 12)
ax.clabel(pt, pt.levels, inline=True, fontsize=10)
ax.scatter(GS_Y, GS_X, linewidths = 10,  color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS VDOP sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optHDOP_VDOP.png')
plt.show()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contour(Lon, Lat, HDOP, zdir='z', offset= -10, cmap=cm.jet, levels = 20, vmax = 350)
ax.clabel(pt, pt.levels, inline=True, fontsize=10)
ax.scatter(GS_Y, GS_X,  linewidths = 10, color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS HDOP sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optHDOP_HDOP.png')
plt.show()

### Зона видимости оптимального HDOP

In [ ]:
class_HDOP = BinByDOP(HDOP)


In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contourf(Lon, Lat, class_HDOP, zdir='z', offset= -10)
ax.scatter(GS_Y, GS_X,  linewidths = 10, color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS HDOP sigma = 12' + '; Cells = ' + str(CalcSquare(HDOP)) + '; Square = ' + str(round(CalcRealSquare(Lat, Lon, HDOP, h),2)) + ' km^2')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optHDOP_HDOP_filter.png')
plt.show()

In [ ]:
class_VDOP = BinByDOP(VDOP)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contourf(Lon, Lat, class_VDOP, zdir='z', offset= -10)
ax.scatter(GS_Y, GS_X,  linewidths = 10, color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS VDOP sigma = 12'  + '; Cells = ' + str(CalcSquare(VDOP)) + '; Square = ' +  str(round(CalcRealSquare(Lat, Lon, VDOP, h),2)) + ' km^2')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optHDOP_VDOP_filter.png')
plt.show()

## Оптимум для VDOP

In [ ]:
#Optimum VDOP
offset_C = max_VDOP[1]
GS_X = np.array([X_c + offset_C * cos(315 * pi / 180), X_c + offset_C * cos(225 * pi / 180), X_c, X_c + offset_C * cos(135 * pi / 180), X_c + offset_C * cos(45 * pi / 180)]) 
GS_Y = np.array([Y_c + offset_C * sin(315 * pi / 180), Y_c + offset_C * sin(225 * pi / 180), Y_c, Y_c + offset_C * sin(135 * pi / 180), Y_c + offset_C * sin(45 * pi / 180)] ) 
HDOP, VDOP = GetHVDOP(Lat, Lon, GS_X, GS_Y)

In [ ]:
%matplotlib inline
plt.scatter(GS_X, GS_Y)
plt.title('VDOP offset = ' + str(offset_C)+ '; Max offset = ' + str(n * step_))

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contour(Lon, Lat, VDOP, zdir='z', offset= -10, cmap=cm.jet, levels = 12)
ax.clabel(pt, pt.levels, inline=True, fontsize=10)
ax.scatter(GS_Y, GS_X, linewidths = 10,  color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS VDOP sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optVDOP_VDOP.png')
plt.show()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contour(Lon, Lat, HDOP, zdir='z', offset= -10, cmap=cm.jet, levels = 20, vmax = 350)
ax.clabel(pt, pt.levels, inline=True, fontsize=10)
ax.scatter(GS_Y, GS_X,  linewidths = 10, color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS HDOP sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optVDOP_HDOP.png')
plt.show()

### Зона видимости оптимального VDOP

In [ ]:
class_HDOP = BinByDOP(HDOP)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contourf(Lon, Lat, class_HDOP, zdir='z', offset= -10)
ax.scatter(GS_Y, GS_X,  linewidths = 10, color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS HDOP sigma = 12'  + '; Cells = ' + str(CalcSquare(HDOP)) + '; Square = ' +  str(round(CalcRealSquare(Lat, Lon, HDOP, h),2)) + ' km^2')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optVDOP_HDOP_filter.png')
plt.show()

In [ ]:
class_VDOP = BinByDOP(VDOP)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contourf(Lon, Lat, class_VDOP, zdir='z', offset= -10)
ax.scatter(GS_Y, GS_X,  linewidths = 10, color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS VDOP sigma = 12'  + '; Cells = ' + str(CalcSquare(VDOP)) + '; Square = ' +  str(round(CalcRealSquare(Lat, Lon, VDOP, h),2)) + ' km^2')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optVDOP_VDOP_filter.png')
plt.show()

# Зависимость от длины базы станций

In [ ]:
delta = []
offset_C = 0
for i in range(n):
    offset_C += step_
    GS_X = np.array([X_c + offset_C * cos(315 * pi / 180), X_c + offset_C * cos(225 * pi / 180), X_c, X_c + offset_C * cos(135 * pi / 180), X_c + offset_C * cos(45 * pi / 180)]) 
    delta.append(GetDiffBetweenGS(GS_X))

In [ ]:
delta

In [ ]:
'''%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
plt.plot(delta, Hcells, label = 'HDOP Cells')
plt.plot(delta, Vcells, label = 'VDOP Cells')
ax.set_title('Cell Count sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(50))
ax.yaxis.set_minor_locator(MultipleLocator(50))
ax.grid('minor')
plt.legend()
plt.xlabel('km')
plt.ylabel('Count')
plt.savefig('Plots/CellsByDelta.png')
plt.show()
'''

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
plt.plot(delta, Hsqueare, label = 'HDOP sqaure')
plt.plot(delta, Vsqueare, label = 'VDOP sqaure')
ax.set_title('Square sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(50))
ax.yaxis.set_minor_locator(MultipleLocator(10000))
ax.grid('minor')
plt.legend()
plt.xlabel('km')
plt.ylabel('km^2')
plt.savefig('Plots/SquareByDelta.png')
plt.show()

In [ ]:

%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
plt.plot(delta, Hsqueare, label = 'HDOP sqaure')
ax.set_title('Square sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(50))
#ax.yaxis.set_minor_locator(MultipleLocator(10000))
ax.grid('minor')
plt.legend()
plt.xlabel('km')
plt.ylabel('km^2')
plt.savefig('Plots/SquareByDeltaHDOP.png')
plt.show()


In [ ]:
'''
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
plt.plot(delta, Zone, label = 'HDOP Zone sqaure')
ax.set_title('Square sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(50))
#ax.yaxis.set_minor_locator(MultipleLocator(10000))
ax.grid('minor')
plt.legend()
plt.xlabel('km')
plt.ylabel('km^2')
plt.savefig('Plots/SquareByDeltaHDOPZone.png')
plt.show()
'''

In [ ]:
'''
index_max_HDOP = np.argmax(Zone)
delta[index_max_HDOP]
'''

In [ ]:

%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
plt.plot(delta, Vsqueare, label = 'VDOP sqaure')
ax.set_title('Square sigma = 12')
ax.xaxis.set_minor_locator(MultipleLocator(50))
#ax.yaxis.set_minor_locator(MultipleLocator(10000))
ax.grid('minor')
plt.legend()
plt.xlabel('km')
plt.ylabel('km^2')
plt.savefig('Plots/SquareByDeltaVDOP.png')
plt.show()


In [ ]:

index_max_VDOP = np.argmax(Vsqueare)
index_max_HDOP = np.argmax(Hsqueare)


In [ ]:
delta[index_max_VDOP]

In [ ]:
delta[index_max_HDOP]

### Нахождение зоны обслуживания оптимальной конфигурации

In [ ]:
#Optimum HDOP
h = 0.005
X_c = 0
Y_c = 39
Lat = np.arange(X_c - 2, X_c + 2, h)
Lon = np.arange(Y_c - 2, Y_c + 2, h) 
Lat, Lon = np.meshgrid(Lat, Lon)
k = len(Lat)
l = len(Lat[0])

offset_C = max_HDOP[1]

GS_X = np.array([X_c + offset_C * cos(315 * pi / 180), X_c + offset_C * cos(225 * pi / 180), X_c, X_c + offset_C * cos(135 * pi / 180), X_c + offset_C * cos(45 * pi / 180)]) 
GS_Y = np.array([Y_c + offset_C * sin(315 * pi / 180), Y_c + offset_C * sin(225 * pi / 180), Y_c, Y_c + offset_C * sin(135 * pi / 180), Y_c + offset_C * sin(45 * pi / 180)] ) 

E = ServiceZone(Lat, Lon, GS_X, GS_Y, h)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
pt = ax.contourf(Lon, Lat, E, zdir='z', offset= -10)
ax.scatter(GS_Y, GS_X,  linewidths = 10, color = 'black', marker = '^')
fig.colorbar(pt)
ax.set_title('GPS HDOP sigma = 12'  + '; Cells = ' + str(CalcSquare(HDOP)) + '; Square = ' +  str(round(CalcRealSquareByBinMat(Lat, Lon, E, h),2)) + ' km^2')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.grid('minor')
plt.savefig('Plots/optVDOP_HDOP_filter_service.png')
plt.show()

In [ ]:
GetDiffBetweenGS([GS_X[0], GS_X[1]])